<a href="https://colab.research.google.com/github/Javcm/Deep-Learning/blob/main/Music_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Considera un corpus de obras (cortas) de piano en formato `MIDI` como las que se encuentran en la carpeta `midi_train`.

In [ ]:
import pickle
import numpy
import os
import keras
from music21 import converter, instrument, note, chord, stream
from IPython.display import Image
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM,Activation,BatchNormalization
from keras.utils import np_utils, plot_model
from keras.callbacks import ModelCheckpoint
from google.colab import drive 
drive.mount('/content/gdrive')


#os.chdir('/home/victor/cursos/optativa2020/')
# el directorio con el corpus de entrenamiento
midi_songs = '/content/gdrive/My Drive/T4 DL/classic_piano_corpus/midi_train'

Mounted at /content/gdrive


Pongo en la página unos MIDIs de entrenamiento, pero lo que recomiendo es que cada quien forme su corpus de entrenamiento. Yo obtuve los midi de http://www.piano-midi.de/midicoll.htm pero también puedes buscar otra fuente.

La parte que sigue puede usarse para el proceso de entrenamiento

In [ ]:
def get_notes():
    """ Obtiene las notas y acordes de los archivos midi que se encuentran en el directorio /midi_songs """
    notes = []

    for file in os.scandir(midi_songs):
        print(file.path)
        midi = converter.parse(file.path)

        print("Analizando %s" % file.name)

        notes_to_parse = None

        # en esta parte, se extraen todas las notas del archivo midi
        # como 'midi' es un objeto Score (que a su vez hereda de un Stream),
        # pueden accederse a todas las partes de la partitura (Score)
        
        # se consideran los casos en que el archivo midi tenga partes que correspondan a diferentes instrumentos
        # (solo se usa el primero, aunque puedan haber mas de 1), o que tenga solo las notas
        midi_parts = instrument.partitionByInstrument(midi)
        if midi_parts: 
            notes_to_parse = midi_parts.parts[0].recurse() 
        else: # notas en estructura flat
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    # guarda las notas. Este paso es muy importante, ya que se usaran en la fase de test,
    # es decir, cuando se generan notas a partir del modelo entrenado
    with open('/content/gdrive/My Drive/T4 DL/data_mid/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes


def prepare_sequences(notes, n_vocab, sequence_length):
    """ Prepara las secuencias a usar en el modelo """

    # nuestro 'vocabulario' consiste en todas las notas unicas de los archivos midi
    pitchnames = sorted(set(item for item in notes))

     # se crea un diccionario para mapear notas a enteros
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # crea las secuencias de entrada y las salidas
    # podria usarse para una arquitectura 'many to one', pero también puedes
    # hacer un stack de RNNs (many to many)
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # redimensiona las secuencias de entrada para la RNN a usar
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normaliza
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [ ]:
notes = get_notes()

In [ ]:
# puedes probar con varios longitudes de secuencias...
sequence = 50
# cantidad de notas
n_vocab = len(set(notes))
print('vocabulario:', n_vocab)

vocabulario: 416


In [ ]:
import numpy as np
network_input, network_output = prepare_sequences(notes, n_vocab, sequence)
print(np.shape(network_input),np.shape(network_output))

(154379, 50, 1) (154379, 416)


No pongo el código del modelo que usé, pero en mi caso, usé la siguiente arquitectura. Puedes probar con ésa misma o alguna otra que tú sugieras.

In [ ]:
model = Sequential()
model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
#plot_model(model,show_shapes=True)
model.summary()

In [ ]:
#Image(retina=True, filename='/home/victor/cursos/figs/midi_LSTM.png')
filepath = "/content/gdrive/My Drive/T4 DL/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"  
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=100, batch_size=64, callbacks=callbacks_list)

Epoch 1/100
2413/2413 [==============================] - 61s 25ms/step - loss: 4.7004
Epoch 2/100
2413/2413 [==============================] - 64s 27ms/step - loss: 4.6503
Epoch 3/100
2413/2413 [==============================] - 64s 26ms/step - loss: 4.6253
Epoch 4/100
2413/2413 [==============================] - 64s 26ms/step - loss: 4.5744
Epoch 5/100
2413/2413 [==============================] - 64s 26ms/step - loss: 4.5060
Epoch 6/100
2413/2413 [==============================] - 64s 27ms/step - loss: 4.4117
Epoch 7/100
2413/2413 [==============================] - 64s 27ms/step - loss: 4.3115
Epoch 8/100
2413/2413 [==============================] - 64s 27ms/step - loss: 4.2027
Epoch 9/100
2413/2413 [==============================] - 64s 26ms/step - loss: 4.0756
Epoch 10/100
2413/2413 [==============================] - 64s 26ms/step - loss: 3.9643
Epoch 11/100
2413/2413 [==============================] - 64s 27ms/step - loss: 3.8591
Epoch 12/100
2413/2413 [============================

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights('saved_model/my_model_weights') 
model.save("/content/gdrive/My Drive/T4 DL/my_model.hdf5")
print("Saved model to disk")

Saved model to disk


Ya que tienes el modelo entrenado y guardado, puedes generar una secuencia y guardarlo en archivo MIDI con el siguiente código

In [ ]:
def prepare_sequences_test(notes, pitchnames, n_vocab):
    """ Prepara las secuencias para usar en el modelo entrenado """
    
    # aqui, se realiza un mapeo (con diccionario) entre las notas y valores enteros y al reves
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # como antes, redimensionamos las entradas para poder usarse con LSTM
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalizamos la entrada
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def generate_notes_test(model, network_input, pitchnames, n_vocab):
    """ Genera notas musicales a partir de una red neuronal basado en una secuencia inicial de notas """
    
    # selecciona una secuencia aleatoria del input a partir de la cual se realizaran predicciones
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # genera 500 notas (puedes cambiarlo)
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output, midfile):
    """ genera un archivo MIDI a partir de las notas generadas (o predichas) """
    offset = 0
    output_notes = []

    # crea los objetos fundamentales Note y Chrod basado en los valores genrados por el modelo entrenado
    for pattern in prediction_output:
        # si las notas forman un acorde...
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # si son notas...
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # incremental el offset en cada iteracion. El offset es la posicion dentro del stream de musica
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp = midfile)


Suponiendo que ya creaste un modelo (my_model), lo entrenaste y LO GUARDASTE, el siguiente código genera la música a partir de ese modelo

In [ ]:
""" Genera las notas y el archivo MIDI (piano) """

# carga las notas usadas cuando se entreno el modelo
with open('/content/gdrive/My Drive/T4 DL/data_mid/notes', 'rb') as filepath:
    notes = pickle.load(filepath)

# Obtiene el nombre de todas las notas (pitches)
pitchnames = sorted(set(item for item in notes))
n_vocab = len(set(notes))

""" Obtiene las secuencias Genera las notas y el archivo MIDI (piano) """
network_input, normalized_input = prepare_sequences_test(notes, pitchnames, n_vocab)

# carga el modelo entrenado
from keras.models import load_model
model = load_model('/content/gdrive/My Drive/T4 DL/my_model.hdf5')
# genera las notas
prediction_output = generate_notes_test(model, network_input, pitchnames, n_vocab)
np.save("/content/gdrive/My Drive/T4 DL/predicted_notes",prediction_output)
# crea el MIDI
create_midi(prediction_output, '/content/gdrive/My Drive/T4 DL/test_output.mid')